<a href="https://colab.research.google.com/github/alexdatadesign/video_whisper/blob/main/video_transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
youtube_video_url = 'https://www.youtube.com/watch?v=dDih3wTuhcM'

In [2]:
# What type of GPU we have here ?
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-dc82b877-8f91-cbd6-6199-d0204466b9b7)


In [3]:
!nvidia-smi

Wed Jan 10 12:34:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install yt-dlp
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-hxagi22z
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-hxagi22z
  Resolved https://github.com/huggingface/transformers.git to commit ffd3710391c0700a3957f0cdf2c99bc5ae966c70
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
import yt_dlp
import subprocess
from pathlib import Path
from IPython.display import display, Markdown, YouTubeVideo

In [6]:
# download youtube video
# extract audio id wav format

ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'outtmpl': '%(id)s.%(ext)s',
        'postprocessors': [{  # Extract audio using ffmpeg
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }]
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download([youtube_video_url])
    list_video_info = [ydl.extract_info(youtube_video_url, download=False)]

v_list = []
for video_info in list_video_info:
    v_list.append(Path(f"{video_info['id']}.wav"))

[youtube] Extracting URL: https://www.youtube.com/watch?v=dDih3wTuhcM
[youtube] dDih3wTuhcM: Downloading webpage
[youtube] dDih3wTuhcM: Downloading ios player API JSON
[youtube] dDih3wTuhcM: Downloading android player API JSON
[youtube] dDih3wTuhcM: Downloading m3u8 information
[info] dDih3wTuhcM: Downloading 1 format(s): 140
[download] Destination: dDih3wTuhcM.m4a
[download] 100% of   13.61MiB in 00:00:00 at 34.43MiB/s  
[FixupM4a] Correcting container of "dDih3wTuhcM.m4a"
[ExtractAudio] Destination: dDih3wTuhcM.wav
Deleting original file dDih3wTuhcM.m4a (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=dDih3wTuhcM
[youtube] dDih3wTuhcM: Downloading webpage
[youtube] dDih3wTuhcM: Downloading ios player API JSON
[youtube] dDih3wTuhcM: Downloading android player API JSON
[youtube] dDih3wTuhcM: Downloading m3u8 information


In [7]:
for video_path_local in v_list:
    if video_path_local.suffix == ".mp4":
        video_path_local = video_path_local.with_suffix(".wav")
        result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])


In [8]:
# transcribe with the latest whisper model

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

sample = str(v_list[0])

result = pipe(sample, return_timestamps=True)
print(result["chunks"])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[{'timestamp': (0.0, 5.42), 'text': " In this video, I'm going to walk you through five simple steps to sell SaaS software as a service."}, {'timestamp': (5.84, 9.52), 'text': ' And you want to make sure you watch this video all the way through because these strategies are'}, {'timestamp': (9.52, 14.9), 'text': " going to be applicable if you are selling a SaaS idea that's not quite built yet and you want to"}, {'timestamp': (14.9, 20.02), 'text': " pre-sell it. And it's also going to work even if you are working at a company selling an established"}, {'timestamp': (20.02, 28.68), 'text': " SaaS product. What's going on, everybody? It's Patrick Dang here. Before we get started, make sure to give this video a like, subscribe, and turn on notifications if you want to see more"}, {'timestamp': (28.68, 33.36), 'text': " sales training videos like this. And let's go ahead and dive right in. Now to understand from"}, {'timestamp': (33.36, 38.54), 'text': ' a high level perspective, the SaaS 

In [9]:
result["text"]

" In this video, I'm going to walk you through five simple steps to sell SaaS software as a service. And you want to make sure you watch this video all the way through because these strategies are going to be applicable if you are selling a SaaS idea that's not quite built yet and you want to pre-sell it. And it's also going to work even if you are working at a company selling an established SaaS product. What's going on, everybody? It's Patrick Dang here. Before we get started, make sure to give this video a like, subscribe, and turn on notifications if you want to see more sales training videos like this. And let's go ahead and dive right in. Now to understand from a high level perspective, the SaaS sales funnel, essentially what we want to be thinking about is thinking our sales process as a funnel, almost like a triangle, right? So at the top of the funnel is your first step where you're gonna obviously have more people and you try to get as many people in your funnel as possible. 

In [10]:
# Store transcripts to GDrive

from google.colab import drive

drive_mount_path = Path("/") / "content" / "drive"
drive.mount(str(drive_mount_path))
drive_mount_path /= "My Drive"
drive_path = "Colab Notebooks/test_transcription"

drive_whisper_path = drive_mount_path / Path(drive_path.lstrip("/"))
drive_whisper_path.mkdir(parents=True, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
with open(str(drive_whisper_path) + "/video_text.txt", "w") as f:
  f.write(result["text"])

In [12]:
import pickle

with open(str(drive_whisper_path) + "/video_ts.pkl", "wb") as f:
    pickle.dump(result["chunks"], f)